Open the bash container:

In [ ]:
img_path=/dodrio/scratch/projects/starting_2022_023/benchmark/scatac_benchmark/0_resources/vsn_cache/vibsinglecellnf-singlecelltoolkit-2022-07-07-0638c1d.img
singularity exec --cleanenv -H $PWD -B /dodrio,/readonly/dodrio,/tmp $img_path bash

Generate fragments:

In [2]:
export POLARS_MAX_THREADS=8
export OMP_NUM_THREADS=8

subset=UCS
for file in libds_preprocessing_out/data/reports/barcard/*$subset*tomerge.tsv
do
    echo $file
    sample=(`basename $file`)
    sample=${sample%.tomerge.tsv}
    # echo $sample
    jaccardfile=$file
    infile=libds_preprocessing_out/data/fragments/$sample.fragments.raw.tsv.gz
    outfile=libds_preprocessing_out/data/fragments/$sample.fragments.tsv
    outfile2=$outfile.gz

    if [ -f "$outfile" ]; then
        echo "$outfile exists, skipping"
    elif [ -f "$outfile2" ]; then
        echo "$outfile2 exists, skipping"
    else 
        echo "$outfile does not exist, generating!"
        merge_cbs_over_jaccard_index_threshold.py -j $jaccardfile -i $infile -o $outfile &
    fi
done

full_preprocessing_out/data/fragments/BIO_ddseq_1.tomerge.tsv
full_preprocessing_out/data/fragments/BIO_ddseq_1.fragments.tsv does not exist, generating!
full_preprocessing_out/data/fragments/BIO_ddseq_2.tomerge.tsv
full_preprocessing_out/data/fragments/BIO_ddseq_2.fragments.tsv.gz exists, skipping
full_preprocessing_out/data/fragments/BIO_ddseq_3.tomerge.tsv
full_preprocessing_out/data/fragments/BIO_ddseq_3.fragments.tsv.gz exists, skipping
full_preprocessing_out/data/fragments/BIO_ddseq_4.tomerge.tsv
full_preprocessing_out/data/fragments/BIO_ddseq_4.fragments.tsv.gz exists, skipping
full_preprocessing_out/data/fragments/BRO_mtscatac_1.tomerge.tsv
full_preprocessing_out/data/fragments/BRO_mtscatac_1.fragments.tsv.gz exists, skipping
full_preprocessing_out/data/fragments/BRO_mtscatac_2.tomerge.tsv
full_preprocessing_out/data/fragments/BRO_mtscatac_2.fragments.tsv.gz exists, skipping
full_preprocessing_out/data/fragments/CNA_10xmultiome_1.tomerge.tsv
full_preprocessing_out/data/fragment

Then bgzip the output:

In [5]:
module load HTSlib
for file in libds_preprocessing_out/data/fragments/*.fragments.tsv
do
    echo $file
    bgzip -@ 8 $file &
done

full_preprocessing_out/data/fragments/BIO_ddseq_2.fragments.tsv
full_preprocessing_out/data/fragments/BIO_ddseq_3.fragments.tsv
full_preprocessing_out/data/fragments/BIO_ddseq_4.fragments.tsv
full_preprocessing_out/data/fragments/BRO_mtscatac_1.fragments.tsv
full_preprocessing_out/data/fragments/BRO_mtscatac_2.fragments.tsv
full_preprocessing_out/data/fragments/CNA_10xmultiome_1.fragments.tsv
full_preprocessing_out/data/fragments/CNA_10xmultiome_2.fragments.tsv
full_preprocessing_out/data/fragments/CNA_10xv11_1.fragments.tsv
full_preprocessing_out/data/fragments/CNA_10xv11_2.fragments.tsv
full_preprocessing_out/data/fragments/CNA_10xv11_3.fragments.tsv
full_preprocessing_out/data/fragments/CNA_10xv11_4.fragments.tsv
full_preprocessing_out/data/fragments/CNA_10xv11_5.fragments.tsv
full_preprocessing_out/data/fragments/CNA_10xv2_1.fragments.tsv
full_preprocessing_out/data/fragments/CNA_10xv2_2.fragments.tsv
full_preprocessing_out/data/fragments/CNA_ddseq_1.fragments.tsv
full_preprocessin

In [ ]:
module load HTSlib
subset=OHS
for file in libds_preprocessing_out/data/fragments/*$subset*.fragments.tsv.gz
do
    echo $file
    if [ -f "$file.gz.tbi" ]; then
        echo exists
    else
        tabix -p bed $file &
    fi
done

And add a new barcode tag `DB` to the corresponding .bam files, based on the barcode translate files

In [7]:
prog=../0_resources/single_cell_toolkit_rust/target/release/add_new_bc_tag_if_old_found
subset=UCS
for file in libds_preprocessing_out/data/bam/*$subset*possorted.bam
do
    echo $file
    sample=(`basename $file`)
    sample=${sample%_____R1.bwa.out.possorted.bam}
    echo $sample
    outfile=libds_preprocessing_out/data/bam/$sample.possorted.dbfixed.bam
    bc_map=libds_preprocessing_out/data/fragments/$sample.fragments.tsv.original_CB_to_merged_CBs.tsv
    ls $bc_map
    if [ -f "$outfile" ]; then
        echo "$outfile exists, skipping"
    else 
        echo "$outfile does not exist, generating!"
        $prog $file $outfile $bc_map CB DB &
    fi
done

full_preprocessing_out/data/bam/BIO_ddseq_1.FULL_____R1.bwa.out.possorted.bam
BIO_ddseq_1.FULL
full_preprocessing_out/data/fragments/BIO_ddseq_1.FULL.fragments.tsv.original_CB_to_merged_CBs.tsv
full_preprocessing_out/data/bam/BIO_ddseq_1.FULL.possorted.dbfixed.bam does not exist, generating!



In [1]:
for file in libds_preprocessing_out/data/bam/*.possorted.dbfixed.bam
do
    echo $file
    outfile=$file.bai
    if [ -f "$outfile" ]; then
        echo "$outfile exists, skipping"
    else 
        echo "$outfile does not exist, generating!"
        samtools index -@ 4 $file &
    fi
done

libds_preprocessing_out/data/bam/BIO_ddseq_1.LIBDS.possorted.dbfixed.bam
libds_preprocessing_out/data/bam/BIO_ddseq_1.LIBDS.possorted.dbfixed.bam.bai exists, skipping
libds_preprocessing_out/data/bam/BIO_ddseq_2.LIBDS.possorted.dbfixed.bam
libds_preprocessing_out/data/bam/BIO_ddseq_2.LIBDS.possorted.dbfixed.bam.bai exists, skipping
libds_preprocessing_out/data/bam/BIO_ddseq_3.LIBDS.possorted.dbfixed.bam
libds_preprocessing_out/data/bam/BIO_ddseq_3.LIBDS.possorted.dbfixed.bam.bai exists, skipping
libds_preprocessing_out/data/bam/BIO_ddseq_4.LIBDS.possorted.dbfixed.bam
libds_preprocessing_out/data/bam/BIO_ddseq_4.LIBDS.possorted.dbfixed.bam.bai exists, skipping
libds_preprocessing_out/data/bam/BRO_mtscatac_1.LIBDS.possorted.dbfixed.bam
libds_preprocessing_out/data/bam/BRO_mtscatac_1.LIBDS.possorted.dbfixed.bam.bai exists, skipping
libds_preprocessing_out/data/bam/BRO_mtscatac_2.LIBDS.possorted.dbfixed.bam
libds_preprocessing_out/data/bam/BRO_mtscatac_2.LIBDS.possorted.dbfixed.bam.bai exi